<a href="https://colab.research.google.com/github/strumberr/fine-tuned-carlsen-model/blob/main/Current_Best_Chess_Magnus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers python-chess torch numpy

In [ ]:
import os
from huggingface_hub import login
from google.colab import userdata
import chess.pgn
import torch
import numpy as np
from transformers import GPT2LMHeadModel, AutoTokenizer, Trainer, TrainingArguments, GPT2Model, AutoModel
from torch.utils.data import Dataset
from torch import nn
import torch.nn.functional as F


# # Authenticate with Hugging Face
# hf_token = userdata.get('HF_TOKEN')
# if hf_token:
#     login(token=hf_token)
# else:
#     raise ValueError("HF_TOKEN not found in Colab secrets. Please add it at https://huggingface.co/settings/tokens.")


In [ ]:
def parse_magnus_pgn(pgn_file_path):
    positions = []
    with open(pgn_file_path, 'r', encoding='utf-8') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            magnus_color = "white" if "Carlsen" in game.headers.get("White", "") else "black"
            if "Carlsen" not in game.headers.get("White", "") and "Carlsen" not in game.headers.get("Black", ""):
                continue
            if "blitz" in game.headers.get("Event", "").lower() or "bullet" in game.headers.get("Event", "").lower():
                continue
            board = game.board()
            node = game
            while node.variations:
                next_node = node.variation(0)
                if (board.turn == chess.WHITE and magnus_color == "white") or \
                   (board.turn == chess.BLACK and magnus_color == "black"):
                    positions.append((board.fen(), next_node.move.uci()))
                board.push(next_node.move)
                node = next_node
    return positions


def tokenize_fen(fen, tokenizer):
    tokens = tokenizer(fen, padding="max_length", max_length=64, truncation=True, return_tensors="pt")
    return tokens

def encode_move(move_str):
    move = chess.Move.from_uci(move_str)
    return move.from_square * 64 + move.to_square


class ChessDataset(Dataset):
    def __init__(self, fens, moves, tokenizer):
        self.fens = fens
        self.moves = [encode_move(move) for move in moves]
        self.tokenizer = tokenizer
        self.tokenized_data = [tokenize_fen(fen, tokenizer) for fen in fens]

    def __len__(self):
        return len(self.fens)

    def __getitem__(self, idx):
        tokens = self.tokenized_data[idx]
        return {
            "input_ids": tokens["input_ids"].squeeze(0),
            "attention_mask": tokens["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.moves[idx], dtype=torch.long)
        }


class ChessMoveClassifier(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.base_model = AutoModel.from_pretrained(model_name)
        self.classifier = nn.Linear(self.base_model.config.hidden_size, 4096)  # 64x64

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
      outputs = self.base_model(
          input_ids=input_ids,
          attention_mask=attention_mask,
          token_type_ids=token_type_ids  # pass it to the base model anyway
      )
      hidden_state = outputs.last_hidden_state[:, 0, :]  # use [CLS] token
      logits = self.classifier(hidden_state)

      if labels is not None:
          loss = F.cross_entropy(logits, labels)
          return {"loss": loss, "logits": logits}
      return {"logits": logits}


In [ ]:
def main():
    model_name = "austindavis/ChessGPT_d12"
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = ChessMoveClassifier(model_name)
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(f"Using device: {device}")

    train_positions = parse_magnus_pgn("/content/carlsen-train.pgn")
    val_positions = parse_magnus_pgn("/content/carlsen-val.pgn")
    print(f"Train positions: {len(train_positions)} | Val positions: {len(val_positions)}")

    model.base_model.gradient_checkpointing_enable()

    train_fens, train_moves = zip(*train_positions)
    val_fens, val_moves = zip(*val_positions)

    train_dataset = ChessDataset(train_fens, train_moves, tokenizer)
    # train_dataset = torch.utils.data.Subset(train_dataset, range(100))  # Using a smaller subset for faster test runs
    val_dataset = ChessDataset(val_fens, val_moves, tokenizer)

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        warmup_steps=100,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        fp16=torch.cuda.is_available(),
        report_to="wandb"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    print("Starting training...")
    trainer.train()



    import os
    os.makedirs("/content/fine_tuned_chessgpt2", exist_ok=True)
    torch.save(model.state_dict(), "/content/fine_tuned_chessgpt2/model.pt")
    tokenizer.save_pretrained("/content/fine_tuned_chessgpt2")
    print("Model saved to /content/fine_tuned_chessgpt2")

    print("Evaluating on validation set...")
    predictions = trainer.predict(val_dataset)

    logits = predictions.predictions
    true_labels = predictions.label_ids
    predicted_indices = np.argmax(logits, axis=1)
    correct = (predicted_indices == true_labels).sum()
    total = len(true_labels)
    accuracy = correct / total

    print(f"Validation Accuracy: {accuracy:.4f}")

    def predict_magnus_move(fen):
        model.eval()
        inputs = tokenize_fen(fen, tokenizer)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            predicted_index = outputs["logits"].argmax(dim=-1).item()
        from_sq = predicted_index // 64
        to_sq = predicted_index % 64
        move = chess.Move(from_sq, to_sq)
        board = chess.Board(fen)
        if move in board.legal_moves:
            return move.uci()
        legal_moves = list(board.legal_moves)
        return legal_moves[0].uci() if legal_moves else "No legal move"

    fen = "rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1"
    predicted_move = predict_magnus_move(fen)
    print(f"Magnus would play: {predicted_move}")

if __name__ == "__main__":
    main()


In [1]:
!ls

sample_data


### Self-play

In [ ]:
import chess
import chess.pgn
import datetime
import random
import torch
from transformers import AutoTokenizer, AutoConfig, GPT2Model
import torch.nn as nn

# === Load model manually ===

class ChessMoveClassifier(nn.Module):
    def __init__(self, model_name, num_labels=4096):
        super().__init__()
        self.base_model = GPT2Model.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.base_model.config.n_embd, num_labels)

    def forward(self, input_ids, attention_mask=None, **kwargs):  # <-- added **kwargs
      outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
      hidden_state = outputs.last_hidden_state[:, -1, :]  # Use last token
      logits = self.classifier(self.dropout(hidden_state))
      return {"logits": logits}


model_dir = "/content/fine_tuned_chessgpt2"
base_model = "austindavis/ChessGPT_d12"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = ChessMoveClassifier(base_model)
model.load_state_dict(torch.load(f"{model_dir}/model.pt", map_location="cpu"))
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# === Tokenize FEN ===
def tokenize_fen(fen, tokenizer):
    return tokenizer(fen, return_tensors="pt")

# === Start game ===
board = chess.Board()
for _ in range(random.randint(4, 12)):
    legal_moves = list(board.legal_moves)
    if not legal_moves:
        break
    board.push(random.choice(legal_moves))

starting_fen = board.fen()
print(f"Starting self-play game from position:\n{starting_fen}\n")

# === PGN setup ===
game = chess.pgn.Game()
game.headers["Event"] = "ChessGPT Self-Play (Random Start)"
game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d")
game.setup(board)
node = game

# === Self-play ===
turn = 0
max_turns = 100

while turn < max_turns and not board.is_game_over():
    inputs = tokenize_fen(board.fen(), tokenizer)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs)["logits"]
        sorted_indices = torch.argsort(logits, dim=-1, descending=True)[0]

        for idx in sorted_indices:
            from_sq = (idx // 64).item()
            to_sq = (idx % 64).item()
            move = chess.Move(from_sq, to_sq)
            if move in board.legal_moves:
                board.push(move)
                node = node.add_variation(move)
                print(f"{'White' if turn % 2 == 0 else 'Black'} plays: {move.uci()}")
                break
        else:
            print("[!] No valid moves predicted")
            break

    turn += 1

# === Save game ===
game.headers["Result"] = board.result()
with open("self_play_game_random.pgn", "w") as f:
    f.write(str(game))

print("Game saved to self_play_game_random.pgn")


### Base model VS magnus

In [ ]:
import chess
import chess.pgn
import datetime
import torch
import random
from transformers import AutoTokenizer, GPT2Model
import torch.nn as nn

# === Model class ===
class ChessMoveClassifier(nn.Module):
    def __init__(self, model_name, num_labels=4096):
        super().__init__()
        self.base_model = GPT2Model.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.base_model.config.n_embd, num_labels)

    def forward(self, input_ids, attention_mask=None, **kwargs):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state[:, -1, :]
        logits = self.classifier(self.dropout(hidden_state))
        return {"logits": logits}

# === Setup device ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Load tokenizer ===
tokenizer = AutoTokenizer.from_pretrained("austindavis/ChessGPT_d12")

# === Load fine-tuned MagnusGPT ===
magnus_model = ChessMoveClassifier("austindavis/ChessGPT_d12")
magnus_model.load_state_dict(torch.load("/content/fine_tuned_chessgpt2/model.pt", map_location=device))
magnus_model.to(device)
magnus_model.eval()

# === Load base model ===
base_model = ChessMoveClassifier("austindavis/ChessGPT_d12")
base_model.to(device)
base_model.eval()

# === Tokenizer wrapper ===
def tokenize_fen(fen):
    return tokenizer(fen, return_tensors="pt")

# === Start from random legal position ===
board = chess.Board()
for _ in range(random.randint(4, 12)):
    legal_moves = list(board.legal_moves)
    if not legal_moves:
        break
    board.push(random.choice(legal_moves))

starting_fen = board.fen()
print(f"Starting from position:\n{starting_fen}\n")

# === Setup PGN ===
game = chess.pgn.Game()
game.headers["Event"] = "MagnusGPT vs BaseGPT"
game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d")
game.setup(board)
node = game

# === Self-play match ===
turn = 0
max_turns = 100

while turn < max_turns and not board.is_game_over():
    inputs = tokenize_fen(board.fen())
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # White = MagnusGPT, Black = BaseGPT
    model = magnus_model if board.turn == chess.WHITE else base_model

    with torch.no_grad():
        logits = model(**inputs)["logits"]
        sorted_indices = torch.argsort(logits, dim=-1, descending=True)[0]

        for idx in sorted_indices:
            from_sq = (idx // 64).item()
            to_sq = (idx % 64).item()
            move = chess.Move(from_sq, to_sq)
            if move in board.legal_moves:
                board.push(move)
                node = node.add_variation(move)
                print(f"{'White (Magnus)' if board.turn == chess.BLACK else 'Black (Base)'} plays: {move.uci()}")
                break
        else:
            print("[!] No valid moves predicted")
            break

    turn += 1

# === Finalize game ===
game.headers["Result"] = board.result()
with open("magnus_vs_base.pgn", "w") as f:
    f.write(str(game))

print("Game saved to magnus_vs_base.pgn")


### Does the model play like magnus?

In [ ]:
import chess.pgn
import torch
from transformers import AutoTokenizer
from transformers import GPT2Model
import torch.nn as nn
import io

# === Model class ===
class ChessMoveClassifier(nn.Module):
    def __init__(self, model_name, num_labels=4096):
        super().__init__()
        self.base_model = GPT2Model.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.base_model.config.n_embd, num_labels)

    def forward(self, input_ids, attention_mask=None, **kwargs):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state[:, -1, :]
        logits = self.classifier(self.dropout(hidden_state))
        return {"logits": logits}

# === Tokenizer wrapper ===
def tokenize_fen(fen, tokenizer):
    return tokenizer(fen, return_tensors="pt")

# === Load model and tokenizer ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "austindavis/ChessGPT_d12"
tokenizer = AutoTokenizer.from_pretrained("/content/fine_tuned_chessgpt2")
model = ChessMoveClassifier(model_name)
model.load_state_dict(torch.load("/content/fine_tuned_chessgpt2/model.pt", map_location=device))
model.to(device)
model.eval()

# === PGN of real Magnus game (as Black) ===
pgn_text = """[Event "5th OIBM"]
[Site "Bad Wiessee GER"]
[Date "2001.10.27"]
[Round "1"]
[White "Kacheishvili,G"]
[Black "Carlsen,M"]
[Result "1-0"]
[WhiteElo "2583"]
[BlackElo "2072"]
[ECO "E32"]

1.d4 Nf6 2.c4 e6 3.Nc3 Bb4 4.Qc2 O-O 5.a3 Bxc3+ 6.Qxc3 b6 7.Bg5 Bb7 8.e3 c5
9.dxc5 bxc5 10.Ne2 Nc6 11.Ng3 Qa5 12.Bxf6 gxf6 13.Qxa5 Nxa5 14.Nh5 Rfd8 15.O-O-O Kf8
16.Nxf6 Ke7 17.Nh5 Ba6 18.Nf4 Bxc4 19.Bxc4 Nxc4 20.Rhe1 Rab8 21.Re2 d5 22.Rc2 Na5
23.Kb1 c4 24.Ne2 Nb7 25.e4 Nc5 26.exd5 Rxd5 27.Nc3 Rg5 28.f4 Rh5 29.Rd4 Rxh2
30.Rxc4 Nd3 31.Ka2 Rb7 32.Ne4 Rd7 33.Nc5 Nxc5 34.Rxc5 Kf6 35.Ra5 Rb7 36.f5 e5
37.Re2 Re7 38.Ra6+ Kg5 39.f6 Rb7 40.Rxe5+ Kg6 41.Re7 Rb8 42.Raxa7 Rxg2 43.Rab7 Rxb7
44.Rxb7 h5 45.a4 Rg4 46.Ka3 h4 47.Rb8 Rg3+ 48.b3 Kxf6 49.Rh8 Kg5 50.a5 Re3
51.Ka4 Re6 52.b4 f5 53.b5 Re1 54.a6 Ra1+ 55.Kb4 f4 56.Kc5 f3 57.b6 Ra5+ 58.Kd4 Rxa6
59.b7 Rb6 60.b8=Q Rxb8 61.Rxb8 Kf4 62.Rf8+ Kg3 63.Ke3 1-0
"""

# === Parse game ===
game = chess.pgn.read_game(io.StringIO(pgn_text))
board = game.board()
magnus_moves = []
fens = []

# Magnus is Black, so collect FENs before each Black move
for idx, move in enumerate(game.mainline_moves()):
    if idx % 2 == 1:  # Black's turn
        fens.append(board.fen())
        magnus_moves.append(move.uci())
    board.push(move)

# === Run predictions and compare ===
correct = 0
total = len(fens)

for i in range(total):
    fen = fens[i]
    true_move = magnus_moves[i]
    inputs = tokenize_fen(fen, tokenizer)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs)["logits"]
        sorted_indices = torch.argsort(logits, dim=-1, descending=True)[0]

        predicted_move = None
        board = chess.Board(fen)
        for idx in sorted_indices:
            from_sq = (idx // 64).item()
            to_sq = (idx % 64).item()
            move = chess.Move(from_sq, to_sq)
            if move in board.legal_moves:
                predicted_move = move.uci()
                break

        is_correct = (predicted_move == true_move)
        correct += int(is_correct)

        print(f"[{i+1}] True: {true_move} | Predicted: {predicted_move} | {'✅' if is_correct else '❌'}")

# === Accuracy report ===
accuracy = correct / total
print(f"\nMagnus Match Accuracy: {accuracy:.2%} ({correct}/{total})")


### Accuracy test training data with carlsons games and the model

In [ ]:
import chess.pgn
import torch
from transformers import AutoTokenizer, GPT2Model
import torch.nn as nn
from tqdm import tqdm

# === Model class ===
class ChessMoveClassifier(nn.Module):
    def __init__(self, model_name, num_labels=4096):
        super().__init__()
        self.base_model = GPT2Model.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.base_model.config.n_embd, num_labels)

    def forward(self, input_ids, attention_mask=None, **kwargs):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state[:, -1, :]
        logits = self.classifier(self.dropout(hidden_state))
        return {"logits": logits}

# === Tokenizer wrapper ===
def tokenize_fen(fen, tokenizer):
    return tokenizer(fen, return_tensors="pt")

# Load tokenizer and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "austindavis/ChessGPT_d12"
tokenizer = AutoTokenizer.from_pretrained("/content/fine_tuned_chessgpt2")
model = ChessMoveClassifier(model_name)
model.load_state_dict(torch.load("/content/fine_tuned_chessgpt2/model.pt", map_location=device))
model.to(device)
model.eval()

# First pass to count games
with open("/content/carlsen-train.pgn", "r", encoding="utf-8") as f:
    game_count = sum(1 for line in f if line.strip() == "")

# Evaluation loop with live accuracy updates
pgn_path = "/content/carlsen-train.pgn"
total = 0
correct = 0

with open(pgn_path, "r", encoding="utf-8") as f:
    with tqdm(total=game_count, desc="Processing games") as pbar:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            if "Carlsen" not in game.headers.get("White", "") and "Carlsen" not in game.headers.get("Black", ""):
                pbar.update(1)
                continue

            magnus_color = "white" if "Carlsen" in game.headers.get("White", "") else "black"
            board = game.board()

            for idx, move in enumerate(game.mainline_moves()):
                if (magnus_color == "white" and idx % 2 == 0) or (magnus_color == "black" and idx % 2 == 1):
                    fen = board.fen()
                    true_move = move.uci()

                    inputs = tokenize_fen(fen, tokenizer)
                    inputs = {k: v.to(device) for k, v in inputs.items()}

                    with torch.no_grad():
                        logits = model(**inputs)["logits"]
                        sorted_indices = torch.argsort(logits, dim=-1, descending=True)[0]

                        predicted_move = None
                        temp_board = chess.Board(fen)
                        for idxx in sorted_indices:
                            from_sq = (idxx // 64).item()
                            to_sq = (idxx % 64).item()
                            candidate_move = chess.Move(from_sq, to_sq)
                            if candidate_move in temp_board.legal_moves:
                                predicted_move = candidate_move.uci()
                                break

                    total += 1
                    if predicted_move == true_move:
                        correct += 1

                    # Update progress bar description with live accuracy
                    if total % 10 == 0:
                        acc = correct / total
                        pbar.set_postfix_str(f"Acc: {acc:.4f} ({correct}/{total})")

                board.push(move)

            pbar.update(1)

# Final output
accuracy = correct / total if total > 0 else 0
print(f"\nFinal Accuracy: {accuracy:.4f} ({correct}/{total})")


In [ ]:
from huggingface_hub import HfApi, upload_folder
import os

hf_token = userdata.get('HF_TOKEN')
repo_id = "strumber/magnusTransformer"
local_folder = "./fine_tuned_chessgpt2"

api = HfApi()
api.create_repo(repo_id=repo_id, token=hf_token, exist_ok=True)

upload_folder(
    folder_path=local_folder,
    repo_id=repo_id,
    token=hf_token,
    repo_type="model"
)

print("Model & tokenizer pushed to Hugging Face Hub!")